# Segragate file
- Blast only allow num size
- Thus seperate it to several files

In [ ]:
import re

filename = "LWW016_all.fas"

path = "data\\fasta_file\\" + filename

def read_fas_as_list(path):
    pattern = r"[>][^>]+"
    with open (path,"r") as f:
        whole_data = f.read()
        return re.findall(pattern,whole_data)

def part_folder (num = 50000):
    datalist = read_fas_as_list(path)
    count = len(datalist[0])
    item_index = 0
    folder_num = 1
    temptxt = ""

    while item_index < len(datalist):
        count += len(datalist[item_index])
        if count < num:
            temptxt += datalist[item_index]
            item_index += 1
        else:
            newpath = "data\\fasta_file\\output\\" + filename[:-4] + "_" + str(folder_num) + ".fas"
            with open (newpath, "w") as f:
                f.write(temptxt)
            folder_num += 1
            count = len(datalist[item_index])
            temptxt = datalist[item_index]
            item_index += 1

    newpath = "data\\fasta_file\\output\\" + filename[:-4] + "_" + str(folder_num) + ".fas"
    with open (newpath, "w") as f:
                f.write(temptxt)

part_folder ()

# Blastn download as json
- parse json

In [ ]:
import json
import pandas as pd

file_name = "data\\blast_result\\blastoutput.json"

query = []
accesssion = []
identity = []
sciname = []
title = []


with open(file_name) as obj:
    getDatas = json.load(obj)

for getData in getDatas:
    for item in getDatas[getData]:
        query.append((item['report']['results']['search']['query_title']))
        accesssion.append(item['report']['results']['search']['hits'][0]['description'][0]['accession'])
        sciname.append(item['report']['results']['search']['hits'][0]['description'][0]['sciname'])
        title.append(item['report']['results']['search']['hits'][0]['description'][0]['title'])
        identity.append(item['report']['results']['search']['hits'][0]["hsps"][0]['identity']/10)


df = pd.DataFrame([query, accesssion, identity, sciname, title]).transpose()
df.columns = ['query', 'accesssion', 'identity', 'sciname', 'title']
df.to_csv("data\\blast_result\\output\\output.csv",index = False)

In [1]:
from Bio.Blast import NCBIWWW

path = "data\\fasta_file\\output\\LWW016_all_1.fas"
outpath = "data\\blast_result\\blast_output.xml"

def Blast_fasta(path,outpath):

    with open (path) as obj:
        fasta_file = obj.read()

    blast_result = NCBIWWW.qblast("blastn", "nt", fasta_file, megablast=True, hitlist_size=10)
    out = blast_result.read()

    with open(outpath, "w") as output_xml:
        output_xml.write(out)

    blast_result.close()


In [2]:
import os

filelist = os.listdir("data\\fasta_file\\output")
for item in filelist:
    path = "data\\fasta_file\\output\\" + item
    outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
    Blast_fasta(path,outpath)

In [1]:
import os

filelist = os.listdir("data\\fasta_file\\output")

def work(item):
    path = "data\\fasta_file\\output\\" + item
    outpath = "data\\blast_result\\output\\" + item[:-4]+ "_out.xml"
    Blast_fasta(path,outpath)

import tqdm
from multiprocessing import Pool
with Pool(1) as p:
    list(tqdm.tqdm(p.imap(work, filelist), total=len(filelist)))

  0%|          | 0/138 [00:00<?, ?it/s]